In [1]:

import pickle
import torch
import numpy as np
import sklearn
import os
from sklearn import preprocessing

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TF info
import tensorflow as tf
import matplotlib.pyplot as plt
from focal_loss import BinaryFocalLoss

# get all files in the folder
def feature(directory,test=False,clf=None):
    '''
    In this function we first extract all the features: places,audio, action, cast.
    The next step is to stack these features in a horizontal manner
    Once that is done we stack every 2 features since we need to predict the transition 
    This leads to same size for ground truth and the labels
    '''
    files = os.listdir(directory)
    total_files = len(files)  # Calculate total number of files

    j = 0
    for i in range(total_files):
        filename = directory + '/' + files[i]
        f = open(filename, 'rb')
        data = pickle.load(f)
        f.close()

        feat1 = data['place']
        feat1 = feat1.data.numpy()  # convert tensors into numpy arrays for sklearn
        feat1_size = feat1.shape[1]

        feat2 = data['cast']
        feat2 = feat2.data.numpy()
        feat2_size = feat2.shape[1]

        feat3 = data['action']
        feat3 = feat3.data.numpy()
        feat3_size = feat3.shape[1]

        feat4 = data['audio']
        feat4 = feat4.data.numpy()
        feat4_size = feat4.shape[1]

        x = np.hstack((feat1, feat2, feat3, feat4))
        y = data['scene_transition_boundary_ground_truth']

        scaler = preprocessing.MinMaxScaler().fit(x)
        x_scaled = scaler.transform(x)

        # Fold the data set to obtain features from adjoining shots
        N = x.shape[0]  # changed from x_scaled
        x_fold = np.zeros((N - 1, 2 * x.shape[1]))  # increasing by 2*x since we need to stack
        for l in range(N - 1):
            x_fold[l, :] = np.hstack((x_scaled[l, :], x_scaled[l + 1, :]))  # combining every 2 shots
        if test is True:
                print(x_fold.shape, 'Test')
                fav = {'scene_transition_boundary_ground_truth': y, 'scene_transition_boundary_prediction': (clf.predict(x_fold)),
                        'shot_end_frame': data['shot_end_frame'], 'imdb_id': data['imdb_id']}
                pickle.dump(fav, open('data_dir/'+str(data['imdb_id'])+'.pkl', "wb"))
        else:
            if (j == 0):
                X = x_fold
                Y = y
            else:
                X = np.concatenate((X, x_fold), axis=0)
                Y = np.concatenate((Y, y),axis=0)

            j = j + 1
    if test is False:
        
        return X,Y



# code by Tae Hwan Jung(Jeff Jung) @graykode
# Reference : https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/LSTM_Attn.py


In [2]:

directory = 'data'
X,Y = feature(directory)


Trying machine learning models

In [3]:
!python evaluate_sceneseg.py data_dir/

# of IMDB IDs: 27
Scores: {
    "AP": 0.09184545653521352,
    "mAP": 0.09697054551742584,
    "Miou": 0.2900415666153362,


evaluate_sceneseg.py:132: RuntimeWarning: invalid value encountered in double_scalars
  fscore_dict[imdb_id] = 2 * p * r / (p + r)


    "Precision": 0.19795936594406763,
    "Recall": 0.15172587742132168,
    "F1": NaN
}


In [18]:
feature('data2/',True,model)

AttributeError: 'list' object has no attribute 'shape'

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

clf = RandomForestClassifier(min_samples_leaf=3)
print(X.shape)
clf.fit(X,Y)
print(2)



(58296, 7168)
2


In [ ]:
feature('data2/',True,clf)

In [ ]:
!python evaluate_sceneseg.py data_dir/